In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os

import pandas as pd
import torch.nn.functional as F

from torch import Tensor
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import LabelEncoder

In [17]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv', index_col='index')
df_submission = pd.read_csv('sample_submission.csv', index_col='index')

In [18]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score

# Подготовка данных
X = train_data['text'].tolist()
y = train_data['rate'].tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загрузка предобученного токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-small")
model = AutoModelForSequenceClassification.from_pretrained("intfloat/multilingual-e5-small", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Предобработка текста и преобразование в тензоры PyTorch
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
train_dataset = CustomDataset(train_encodings, y_train)
test_dataset = CustomDataset(test_encodings, y_test)

In [20]:
# Параметры обучения модели
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    output_dir='./results',  # Добавляем output_dir
    logging_dir='./logs',
    logging_steps=100,  # Указываем, как часто логировать прогресс
)

# Создание объекта Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)



/Users/muzafarov/anaconda3/envs/NLP/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
trainer.train()

  0%|          | 0/14601 [00:00<?, ?it/s]

In [ ]:
# Оценка качества модели
predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(-1)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Использование модели для предсказаний
new_reviews = [...] # новые отзывы, которые нужно оценить
new_encodings = tokenizer(new_reviews, truncation=True, padding=True)
new_dataset = torch.utils.data.TensorDataset(
    torch.tensor(new_encodings.input_ids),
    torch.tensor(new_encodings.attention_mask),
)
predictions = trainer.predict(new_dataset)
predicted_ratings = predictions.predictions.argmax(-1)
print("Predicted ratings:", predicted_ratings)